<a href="https://colab.research.google.com/github/nhsengland/ds_251_RAG/blob/open_source_rag_clean/open_source_rag_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using RAG with NHS Conditions - Demo

In [1]:
!git clone https://github.com/nhsengland/ds_251_RAG

Cloning into 'ds_251_RAG'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 312 (delta 81), reused 129 (delta 74), pack-reused 159
Receiving objects: 100% (312/312), 27.92 MiB | 21.06 MiB/s, done.
Resolving deltas: 100% (134/134), done.


install relevant dependencies using pip

In [2]:
%cd ds_251_RAG

/content/ds_251_RAG


In [ ]:
%pip install --upgrade --quiet  transformers langchain_community bitsandbytes langchain accelerate tensorflow==2.15 chromadb unstructured sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Prompt needs to be in the format given in the template in order for the phi-2 model to understand the input and generate a correct response

# Setup

In [14]:
import glob
import os
import pandas as pd
import toml


import src.models as models

from tqdm import tqdm

config = toml.load("config.toml")

if config['DEV_MODE']:
    config['PERSIST_DIRECTORY'] += "/dev"

First we initialise the RAG pipeline - this is an object which links the vector-store, and the LLM, so when you pass a query in it get passed back into the database, and then returns the response.

There are also methods for adding documents to the database

In [8]:
rag_pipeline = models.RagPipeline(config['EMBEDDING_MODEL'], config['PERSIST_DIRECTORY'], model_type='phi2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

need to fill the database if it's empty (this might take 5 mins or so the first time, unless you've got a nice graphics card!)

In [26]:
# Add documents if there are non - if in DEV mode, don't add any more (if it's not empty)
if len(rag_pipeline.vectorstore.get()['documents']) == 0 or (not config['DEV_MODE']):
    rag_pipeline.load_documents()

Using the phi2 model generate a response for the question below with RAG turned off

## Generating Response from Cogstack Questions

<h3> Load in Cogstack QA from Github Repo </h3>

link to cogstack QA data "https://raw.githubusercontent.com/CogStack/OpenGPT/main/data/nhs_uk_full/prepared_generated_data_for_nhs_uk_qa.csv"

In [25]:
#load processed questions and answers
cogstack_qa = pd.read_csv('src/model_eval/cogstack_qa_data_process.csv')

#select a random sample of questions
sample_qa = cogstack_qa.sample(n = 5, random_state = 999)
sample_qa

,Unnamed: 0,question,answer,reference,short_reference
24400,24400,What can I do if someone with epilepsy has a s...,If the person is in a wheelchair during a seiz...,https://www.nhs.uk/conditions/what-to-do-if-so...,what-to-do-if-someone-has-a-seizure-fit
16010,16010,What is included in the prenatal and postnatal...,The prenatal and postnatal yoga video is desig...,https://www.nhs.uk/conditions/nhs-fitness-stud...,nhs-fitness-studio
8321,8321,Why would someone need dialysis?,People with advanced chronic kidney disease (k...,https://www.nhs.uk/conditions/dialysis/,dialysis
7153,7153,How can I check if a hospital or clinic is reg...,To check if a hospital or clinic registered to...,https://www.nhs.uk/conditions/cosmetic-procedu...,cosmetic-procedures
17219,17219,What happens if my IUD is causing PID symptoms?,"If your IUD is causing PID symptoms, you may b...",https://www.nhs.uk/conditions/pelvic-inflammat...,pelvic-inflammatory-disease-pid


# Testing Phi-2 model - RAG off

In [20]:
rag_pipeline.llm('Instruct: {} \n\n Output:'.format(question))

KeyboardInterrupt: 

In [18]:
question = sample_qa['question'].values[0]

result = rag_pipeline.answer_question(question, model_type = 'phi2', rag=False)

print(result)

KeyboardInterrupt: 

Now try generating the response with RAG turned on

In [12]:
question = "What are the symptoms of achalasia?"

result = rag_pipeline.answer_question(question, rag=True)

print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Instruction: With this context

docs/achalasia.txt:
Other symptoms include:

bringing back up undigested food
choking and coughing fits
heartburn
chest pain
repeated chest infections
drooling of vomit or saliva
gradual but significant weight loss
Symptoms of achalasia may start at any time of life.

Swallowing problems can also be caused by cancers of the mouth, throat and oesophagus. There’s a link between long-term achalasia and the risk of developing cancer of the oesophagus, but the risk is small.

create a final answer with references ("SOURCES"). If you don't know the answer, just say that you don't know. Don't try to make up an answer. Always return the file name from the context used to answer 
the question

Question: What are the symptoms of achalasia?
Output:

> Finished chain.

> Finished chain.
Instruction: With this context

docs/achalasia.txt:
Other symptoms include:
